In [ ]:
U = RealDistribution('uniform', [0, 1])

In [183]:
#I think this should work for arbitrary matrices. Just make sure the birth_distn and survival parameters
#    have the same length. survival = [s_0, s_1, s_2, . . ., s_\betahat], while 
#    birth_distn = [b_0, b_1, . . ., b_\betahat] (where it ends is actually something I should think about more).

def arbitrary_L(survival, birth_distn):
    if len(survival) != len(birth_distn):
        return('Gimme survival and birth_distn of same length')
    else:
        n = len(survival)
    L = zero_matrix(QQ, n)
    L[0, 0] = 2*survival[0]*birth_distn[0]
    for i in range(1, n):
        L[0, i] = survival[i]*birth_distn[i]
        L[i, i] = survival[i]*birth_distn[i]
    for i in range(n-1):
        L[i+1, i] = survival[i]*(1-birth_distn[i])
    return(L)

def get_leading_eigenvalue(survival, birth_distn = None):
    if birth_distn == None:
        birth_distn = [0]*len(survival)
        birth_distn[-1] = 1
    l = arbitrary_L(survival, birth_distn)
    evals = l.eigenvalues()
    moduli = [abs(e) for e in evals]
    r = max(moduli).n()
#    pos_real = [e for e in evals if e in RR and e>0]
#    if len(pos_real)>1:
#        return "Error: more than one positive, real eigenvalue"
#    else:
#        return pos_real[0]
    return r
get_leading_eigenvalue([1/2, 1/2, 5/6])

#We need a function which returns the optimal \betahat. 
def get_optimal_betahat(survival):
    growth_rates = [0]*len(survival)
    growth_rates[0] = survival[0]*2
    for i in range(1, len(survival)):
        truncated_survival = survival[:i+1]
        ri = get_leading_eigenvalue(truncated_survival)
        growth_rates[i] = ri
    betahat = growth_rates.index(max(growth_rates))
    return(betahat)

def get_rbetahat(survival):
    betahat = get_optimal_betahat(survival)
    rbetahat = get_leading_eigenvalue(survival[:betahat + 1])
    return(rbetahat)

def make_random_survival(n = 10):
    survival = [0]*n
    for i in range(n):
        si = U.get_random_element()
        survival[i] = si
    return(survival)

def make_random_birth_distn(n = 10):
    birth_distn = make_random_survival(n)
    birth_distn[-1] = 1
    return(birth_distn)

def test_single_survival(n = 10, num_tests = 10):
    exceptions = []
    survival = make_random_survival(n)
    rbetahat = get_rbetahat(survival)
    for _ in range(num_tests):
        birth_distn = make_random_birth_distn(n)
        rb = get_leading_eigenvalue(survival, birth_distn)
        if rb > rbetahat:
            exceptions.append(tuple(survival, birth_distn))
    if exceptions == []:
        return('All good; no exceptions :-)')
    else:
        return(exceptions)
    



    

In [188]:
test_single_survival(n = 6, num_tests = 1000)

'All good; no exceptions :-)'

#### An important note: I had assumed that the leading eigenvalue of our matrix L_b would be unique, just as it is for L. But that isn't the case, so instead I'm going to look at the maximal absolute value, which I feel will still be < r. 

#### So now we know what the baseline fitness of our population is, for a specific s_0, s_1, and s_2. The quesiton is: can we obtain a higher fitness by varying b from [0, 0,1]? My instinct is no. Why not? If squirrels are best off by waiting until 2 rather than any other level, then why would they be able to do better by having some parts of their population peel off early and start reproducing? Answer: they wouldn't. But we'll see.

In [104]:
s = [1/2, 1/2, 5/6]
b = [0, 0,1]
r0 = get_leading_eigenvalue(s, b)
exceptions = []

In [120]:
for k in range(1000):
    for i in range(2):
        bi = U.get_random_element()
        b[i] = bi
    r = get_leading_eigenvalue(s, b)
    if r > r0:
        print("Exception found :-O")
        exceptions.append(b)

#### In the case where all savings levels give the same eigenvalue of 1, a numerics problem makes it look like we can do better by peeling off early. But we actually can't.

In [232]:
s = [1/2, 2/3, 3/4]
b = [0, 0,1]
r0 = get_leading_eigenvalue(s, b)

In [233]:
exceptions = []
for k in range(100):
    b = [0, 0, 1]
    for i in range(2):
        bi = U.get_random_element()
        b[i] = bi
    r = get_leading_eigenvalue(s, b)
    if r > r0:
#        print("Exception found :-O")
        exceptions.append(b)
len(exceptions)

28

In [230]:
for e in exceptions:
    print(get_leading_eigenvalue(s, e))

1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?
1.000000000000001?


#### In principle I ought to try with other initial distributions. But that becomes tough, since I need to know that my si do reasonable things. I'm convinced that my instinct was correct, and r_{L_b} \le r_L irrespective of the b

In [7]:
L = L3([1/2, 1/2, 5/6])
v = vector([1, 0, 0])

In [8]:
def growth(n):
    return(L^n*v.n())
def growth_rate(L):
    v = vector([1, 0, 0])
    n0 = sum(L^100*v)
    n1 = sum(L^101*v)
    return(n1/n0.n())

In [34]:
growth_rate(L)

1.02978824866897

In [14]:
get_leading_eigenvalue([1/2, 1/2, 5/6], [0, 1/4, 1])

1.017929961727893?

In [19]:
Lb = L3([1/2, 1/2, 5/6], [0, 49/100, 1])
growth_rate(Lb)

1.00089922027258

In [170]:
s = make_random_survival(10)
b = make_random_birth_distn(10)

In [176]:
%time get_leading_eigenvalue(s, b)

CPU times: user 73.6 ms, sys: 4.05 ms, total: 77.6 ms
Wall time: 75.1 ms


1.01748135389368